In [1]:
import numpy as np
import math
page_accesses = open(r"acroread_trace.100000000","r")

accesses = []
rw = []
for access in page_accesses:
    s = access.split()
    accesses.append(s[3])
    rw.append(s[2])
page_accesses.close()

In [2]:
training = accesses[:math.floor(0.7*len(accesses))] # 0.6 of the data set is the training set 
# verifying = accesses[math.floor(0.6*len(accesses)):math.floor(0.9*len(accesses))]
testing = accesses[math.floor(0.7*len(accesses)):]
print(len(np.unique(training))) # There are 818 unique addresses

size = 64 # Number of page frames

818


In [4]:
# training = ["c", "a", "d", "b", "e", "b", "a", "b", "c", "d"]
# size = 4

# Not actually FIFO
# faults_fifo = 0
# fifo_frames = []
# for p in training: # The first <size> accesses will not fault
#     if p not in fifo_frames:
#         if len(fifo_frames) >= size: 
#             faults_fifo += 1
#             fifo_frames.pop(0)
#         fifo_frames.append(p)
# print(faults_fifo) # Number of faults
# print(100*faults_fifo/len(training)) # Percent of all accesses that resulted in a fault

In [62]:
# Clairvoyant (may be slightly inaccurate for accesses near the end)
faults_cv = 0
cv_frames = []
for i in range(len(testing)): 
    p = testing[i]
    if p not in cv_frames:
        if len(cv_frames) >= size: 
            faults_cv += 1
            missing = cv_frames.copy()
            for j in range(i, len(training)):
                if len(missing) <= 1:
                    break
                if training[j] in missing: # Or accesses[j]
                    missing.remove(training[j])
            
            cv_frames.remove(missing[0])
        cv_frames.append(p)
print(faults_cv) 
print(100*faults_cv/len(testing))

# 34245
# 1.3592424917183255

34245
1.3592424917183255


In [63]:
# FIFO
faults_fifo = 0
fifo_frames = []
pointer = 0
for p in training: # The first <size> accesses will not fault
    if p not in fifo_frames:
        if len(fifo_frames) >= size: 
            faults_fifo += 1
            fifo_frames.pop(pointer)
        fifo_frames.insert(pointer, p)
        pointer = (pointer + 1) % size
print(faults_fifo) # Number of faults
print(100*faults_fifo/len(training)) # Percent of all accesses that resulted in a fault

# 91888
# 3.647191533917754

91888
3.647191533917754


In [6]:
# LRU
faults_lru = 0
lru_frames = []
lru_stack = []
for p in testing: 
    if p not in lru_frames:
        if len(lru_frames) >= size: 
            faults_lru += 1
            lru_frames.remove(lru_stack[-1])
            lru_stack.pop(-1)
        lru_frames.append(p)
    if p in lru_stack: 
        lru_stack.remove(p)
    lru_stack.insert(0, p)
print(faults_lru) # Number of faults
print(100*faults_lru/len(testing)) # Percent of all accesses that resulted in a fault
# 71014
# 2.8186668508361854

71014
2.8186668508361854


In [66]:
# One-handed clock
faults_c1 = 0
c1_frames = []
c1_clock = [] # [used, dirty]
for i in range(size):
    c1_clock.append([0, 0])
c1_pointer = 0
for p in testing:
    if p not in c1_frames:
        while True:
            if c1_clock[c1_pointer][0] == 0:
                if len(c1_frames) >= size: 
                    faults_c1 += 1
                    c1_frames.pop(c1_pointer)
                c1_frames.insert(c1_pointer, p)
                c1_clock[c1_pointer][0] = 1
                break; 
            else:
                c1_clock[c1_pointer][0] = 0
            c1_pointer = (c1_pointer + 1) % size
    else:
        c1_clock[c1_frames.index(p)][0] = 1
print(faults_c1) 
print(100*faults_c1/len(testing))

# 73060
# 2.8998760824920677

73060
2.8998760824920677


In [64]:
# Second Chance (Two-handed clock) (With read/write)
faults_c2 = 0
c2_frames = []
c2_clock = [] # [used, dirty]
for i in range(size):
    c2_clock.append([0, 0])
c2_pointer = 0
for i in range(len(testing)): 
    p = testing[i]
    if p not in c2_frames:
        while True:
            if (c2_clock[c2_pointer][0] == 0) and (c2_clock[c2_pointer][1] == 0):
                if len(c2_frames) >= size: 
                    faults_c2 += 1
                    c2_frames.pop(c2_pointer)
                c2_frames.insert(c2_pointer, p)
                c2_clock[c2_pointer][0] = 1
                c2_clock[c2_pointer][1] = 1 if (rw[j] == "W") else 0
                break; 
            else:
                if c2_clock[c2_pointer][0] == 0:
                    c2_clock[c2_pointer][1] = 0
                c2_clock[c2_pointer][0] = 0                
            c2_pointer = (c2_pointer + 1) % size
    else:
        j = c2_frames.index(p)
        c2_clock[j][0] = 1
        c2_clock[j][1] = 1 if (rw[i] == "W") else 0
print(faults_c2) 
print(100*faults_c2/len(testing))

# 72820
# 2.890350072913665

72820
2.890350072913665


In [12]:
# LFU (ideal)
faults_lfu = 0
lfu_frames = []
lfu_count = {}
for p in testing: 
    if p not in lfu_frames:
        if len(lfu_frames) >= size: 
            faults_lfu += 1
            # lfu_frames.remove(min(lfu_count, key=lfu_count.get))
            minimum = lfu_count.get(lfu_frames[0])
            value = lfu_frames[0]
            # We can't just keep track of the minimum value overall because the page with 
            # the absolute minimum value may not be in the page frames
            for frame in lfu_frames:                
                if minimum > lfu_count.get(frame): 
                    minimum = lfu_count.get(frame)
                    value = frame
            lfu_frames.remove(value)
        lfu_frames.append(p)
    lfu_count.update({p: lfu_count.setdefault(p, 0) + 1})
print(faults_lfu) 
print(100*faults_lfu/len(testing))

# 604907
# 24.009791150178334

604907
24.009791150178334


In [22]:
# Custom: Decaying Least Frequently Used
faults_dlfu = 0
dlfu_frames = []
dlfu_count = {}
for p in testing: 
    if p not in dlfu_frames:
        if len(dlfu_frames) >= size: 
            faults_dlfu += 1
            minimum = dlfu_count.get(dlfu_frames[0])
            value = dlfu_frames[0]
            for frame in dlfu_frames:                
                if minimum > dlfu_count.get(frame): 
                    minimum = dlfu_count.get(frame)
                    value = frame
            dlfu_frames.remove(value)
        dlfu_frames.append(p)
    dlfu_count.update({p: 1})
print(faults_dlfu) 
print(100*faults_dlfu/len(testing))

# 0.1*a*lfu_count.setdefault(p, 0) + 1
# a = -10 to 10
# 582188
# 23.108035268462796
# 1576052
# 62.55619353358593
# 1415281
# 56.17491817554689
# 1241986
# 49.296543884341546
# 771477
# 30.621238714655526
# 995762
# 39.52349312420567
# 930801
# 36.945080173278114
# 484629
# 19.235752066548702
# 831424
# 33.00063744880762
# 374698
# 14.872403070867954
# 91888
# 3.647191533917754
# 95983
# 3.8097290723492487
# 97275
# 3.8610107572463166
# 98819
# 3.9222947522007066
# 100561
# 3.991437705057279
# 103573
# 4.110989125266232
# 106864
# 4.241614531610078
# 112975
# 4.4841705505001554
# 122731
# 4.871402839862222
# 149158
# 5.920335569564082
# 604907
# 24.009791150178334

91888
3.647191533917754


In [16]:
index_to_page = np.unique(training)
page_to_index = {}
for i in range(len(index_to_page)):
    page_to_index.update({index_to_page[i] : i})

recent_index = []
for i in range(len(index_to_page)):
    recent_index.append(len(training))

pick_highest = []
index = len(training) - 1
for p in reversed(training):
    recent_index[page_to_index.get(p)] = index
    pick_highest.append(recent_index.copy())
    index -= 1

In [17]:
# Given an array of past accesses, what will be the (# of page frames)th page called in the future? 
# Inputs: Array of the last x page accesses
# Label: The page in the current page table that will be accessed furthest in the future
# Test for thrashing? Include R/W?

trainData = []
trainLabel = []

# TLB cache size
past_array = [] # Number of page frames (64) long
past_size = 16 # We take the rightmost accesses of past_array

for i in range(len(training) - size): 
    past_array = training[i : i + size]
    m = -1
    mi = -1
    for j in past_array[size - past_size:]: 
        trainData.append(page_to_index.get(j))
        if m < pick_highest[len(training) - i - 1][page_to_index.get(j)]:
            m = pick_highest[len(training) - i - 1][page_to_index.get(j)]
            mi = page_to_index.get(j)
    trainLabel.append(mi) 
# If there's a page that's in the cache but not in the past_array, remove that first

# Note: We should try finding mi over all of past_array, since those would include more of the pages in cache 
# (while not contain any pages not in the cache)

In [18]:
trainData = np.reshape(np.asarray(trainData[:len(trainData)-size]), (-1, past_size))
# Linear regression based on the a*past_array --> id of the page in the cache that's removed

In [19]:
def linear_regression_ridge(X, Y, lamb):
    N = len(Y)
    X1 = np.hstack((np.ones((N,1)),X))
    trans = np.dot(X1.transpose(),X1)
    # diag = np.diag(np.full(trans.shape[0],lamb))
    # diag[0][0] = 0
    for i in range(1, trans.shape[0]):
        trans[i][i] += lamb 
    betas = np.dot(np.linalg.inv(trans), np.dot(X1.transpose(),Y)) 
    return betas

In [20]:
betasList = []
lambdas = [0.001, 0.1, 10, 1000]
for l in lambdas: 
    betasList.append(linear_regression_ridge(trainData, trainLabel[:len(trainData)], l))
print(betasList)

[array([ 4.96368853e+01, -4.58031081e-03, -1.25909279e-03, -5.01952943e-04,
        2.61980997e-02,  3.30231359e-02,  3.53970029e-02,  4.14137673e-02,
        3.86820554e-02,  5.83073364e-02,  7.17120261e-02,  7.22937465e-02,
        7.78366009e-02,  6.78167121e-02,  1.01691993e-01,  1.33621948e-01,
        2.23922676e-01]), array([ 4.96368853e+01, -4.58031081e-03, -1.25909279e-03, -5.01952943e-04,
        2.61980997e-02,  3.30231359e-02,  3.53970029e-02,  4.14137673e-02,
        3.86820554e-02,  5.83073364e-02,  7.17120261e-02,  7.22937465e-02,
        7.78366009e-02,  6.78167121e-02,  1.01691993e-01,  1.33621948e-01,
        2.23922676e-01]), array([ 4.96368853e+01, -4.58031080e-03, -1.25909279e-03, -5.01952940e-04,
        2.61980997e-02,  3.30231359e-02,  3.53970029e-02,  4.14137673e-02,
        3.86820554e-02,  5.83073364e-02,  7.17120261e-02,  7.22937465e-02,
        7.78366009e-02,  6.78167121e-02,  1.01691993e-01,  1.33621948e-01,
        2.23922676e-01]), array([ 4.96368861e+0

In [21]:
# ML
# If there's a page that's in the cache but not in the past_array, remove that first
for betas in betasList: 
    faults_ML = 0
    faults_ML2 = 0
    ML_frames = []
    prev = []
    for p in testing: 
        if p not in ML_frames: 
            if len(ML_frames) >= size: 
                faults_ML += 1
                removal_index = betas[0]
                for i in range(len(prev)):
                    removal_index += betas[i + 1] * page_to_index.get(prev[i])
                if index_to_page[math.floor(removal_index)] not in ML_frames: 
                    for j in ML_frames: 
                        if j not in prev: 
                            ML_frames.remove(j)
                            break; 
                    # Default to LRU? (Clairvoyant misses)
                else: 
                    faults_ML2 += 1
                    ML_frames.remove(index_to_page[math.floor(removal_index)])
            ML_frames.append(p)
        if len(prev) >= past_size: 
            prev.pop(0)
        prev.append(p)
    print(faults_ML)
    print(100*faults_ML/len(testing)) 
    print(faults_ML2)
    print(100*faults_ML2/len(testing)) 
    print()

# 16 --> 3.599%
# 90699
# 3.5999980947980843
# 5745
# 0.22802885428301298

# 32 --> 3.607%
#90877
#3.607063218568733
#7009
#0.27819917139593353

45165
3.5853518550712904
2856
0.22671902796598262

45165
3.5853518550712904
2856
0.22671902796598262

45165
3.5853518550712904
2856
0.22671902796598262

45165
3.5853518550712904
2856
0.22671902796598262



In [22]:
for betas in betasList: 
    faults_ML2 = 0
    faults_ML2_1 = 0
    ML2_frames = []
    prev = []
    ML2_stack = []
    for p in testing: 
        if p not in ML2_frames: 
            if len(ML2_frames) >= size: 
                faults_ML2 += 1
                removal_index = betas[0]
                for i in range(len(prev)):
                    removal_index += betas[i + 1] * page_to_index.get(prev[i])
                if index_to_page[math.floor(removal_index)] not in ML2_frames: 
                    ML2_frames.remove(ML2_stack[-1])
                    ML2_stack.pop(-1)
                else: 
                    faults_ML2_1 += 1
                    ML2_frames.remove(index_to_page[math.floor(removal_index)])
                    ML2_stack.remove(index_to_page[math.floor(removal_index)])
            ML2_frames.append(p)
        if len(prev) >= past_size: 
            prev.pop(0)
        prev.append(p)
        if p in ML2_stack: 
            ML2_stack.remove(p)
        ML2_stack.insert(0, p)
    print(faults_ML2)
    print(100*faults_ML2/len(testing)) 
    print(faults_ML2_1)
    print(100*faults_ML2_1/len(testing)) 
    print()
# 71929
# 2.8549847623538454

35941
2.8531192521447415
1947
0.1545595054095827

35941
2.8531192521447415
1947
0.1545595054095827

35941
2.8531192521447415
1947
0.1545595054095827

35941
2.8531192521447415
1947
0.1545595054095827



In [23]:
# Modified LRU
trainData = []
trainLabel = []

lruML_frames = []
lruML_stack = []
iterator = 0
for p in training:    
    if len(lruML_stack) >= size: 
        m = -1
        mi = -1
        for j in lruML_stack: 
            trainData.append(page_to_index.get(j))
            if m < pick_highest[iterator][page_to_index.get(j)]:
                m = pick_highest[iterator][page_to_index.get(j)]
                mi = page_to_index.get(j)
        trainLabel.append(mi)
    iterator += 1
    
    if p not in lruML_frames:
        if len(lruML_frames) >= size: 
            lruML_frames.remove(lruML_stack[-1])
            lruML_stack.pop(-1)
        lruML_frames.append(p)
    if p in lruML_stack: 
        lruML_stack.remove(p)
    lruML_stack.insert(0, p)
     
# Stack --> Clairvoyant replacement
# LFU counts --> Clairvoyant replacement

In [24]:
trainData = np.reshape(np.asarray(trainData[:len(trainData)-size]), (-1, size))
betasList = []
lambdas = [0.001, 0.1, 10, 1000]
for l in lambdas: 
    betasList.append(linear_regression_ridge(trainData, trainLabel[:len(trainData)], l))

In [25]:
for betas in betasList: 
    faultsML_lru = 0
    faultsML_lru2 = 0
    lruML_frames = []
    lruML_stack = []
    for p in testing: 
        if p not in lruML_frames:
            if len(lruML_frames) >= size: 
                faultsML_lru += 1

                removal_index = betas[0]
                for i in range(len(lruML_stack)):
                    removal_index += betas[i + 1] * page_to_index.get(lruML_stack[i])
                removal_index = math.floor(removal_index)
                if removal_index >= len(index_to_page) or index_to_page[removal_index] not in lruML_frames: 
                    lruML_frames.remove(lruML_stack[-1])
                    lruML_stack.pop(-1) 
                else: 
                    faultsML_lru2 += 1
                    lruML_frames.remove(index_to_page[removal_index])
                    lruML_stack.remove(index_to_page[removal_index])

            lruML_frames.append(p)
        if p in lruML_stack: 
            lruML_stack.remove(p)
        lruML_stack.insert(0, p)

    print(faultsML_lru) 
    print(100*faultsML_lru/len(testing))
    print(faultsML_lru2) 
    print(100*faultsML_lru2/len(testing))
    print()

# 72760
# 2.8879685705190643
# 7044
# 0.2795883811261172

36296
2.8813003638141823
3346
0.2656169004111267

36296
2.8813003638141823
3346
0.2656169004111267

36296
2.8813003638141823
3346
0.2656169004111267

36296
2.8813003638141823
3346
0.2656169004111267



In [ ]:
######################
#  Scratch programs  #
######################
# They're not bad though in terms of performance

In [23]:
# LFU (decaying 2)
faults_lfu = 0
lfu_frames = []
lfu_count = {}
for p in training: 
    if p not in lfu_frames:
        if len(lfu_frames) >= size: 
            faults_lfu += 1
            # lfu_frames.remove(min(lfu_count, key=lfu_count.get))
            minimum = lfu_count.get(lfu_frames[0])
            value = lfu_frames[0]
            # We can't just keep track of the minimum value overall because the page with 
            # the absolute minimum value may not be in the page frames
            for frame in lfu_frames:                
                if minimum > lfu_count.get(frame): 
                    minimum = lfu_count.get(frame)
                    value = frame
            lfu_frames.remove(value)
        lfu_frames.append(p)
    for k in lfu_count.keys(): 
        lfu_count.update({k : lfu_count.get(k) * 0.5})
    lfu_count.update({p: lfu_count.setdefault(p, 0) + 1})
print(faults_lfu) 
print(100*faults_lfu/len(training))

# 72074
# 2.8607400598074633

72074
2.8607400598074633


In [24]:
# LFU (decaying 3)
faults_lfu = 0
lfu_frames = []
lfu_count = {}
for p in training: 
    if p not in lfu_frames:
        if len(lfu_frames) >= size: 
            faults_lfu += 1
            # lfu_frames.remove(min(lfu_count, key=lfu_count.get))
            minimum = lfu_count.get(lfu_frames[0])
            value = lfu_frames[0]
            # We can't just keep track of the minimum value overall because the page with 
            # the absolute minimum value may not be in the page frames
            for frame in lfu_frames:                
                if minimum > lfu_count.get(frame): 
                    minimum = lfu_count.get(frame)
                    value = frame
            lfu_frames.remove(value)
        lfu_frames.append(p)
    for k in lfu_count.keys(): 
        lfu_count.update({k : max(0, lfu_count.get(k) - 1)})
    lfu_count.update({p: lfu_count.setdefault(p, 0) + 1})
print(faults_lfu) 
print(100*faults_lfu/len(training))
# 93024
# 3.6922813125888596

93024
3.6922813125888596


In [ ]:
# Modified LFU
# After a certain point, it will always recommend the same thing though
lfu_frames = []
lfu_count = {}
iterator = 0
for p in training: 
    if p not in lfu_frames:
        if len(lfu_frames) >= size:             
            minimum = lfu_count.get(lfu_frames[0])
            value = lfu_frames[0]
            for frame in lfu_frames:                
                if minimum > lfu_count.get(frame): 
                    minimum = lfu_count.get(frame)
                    value = frame
            lfu_frames.remove(value)
        lfu_frames.append(p)
    lfu_count.update({p: lfu_count.setdefault(p, 0) + 1})
    iterator += 1
print(faults_lfu) 
print(100*faults_lfu/len(training))